<a href="https://colab.research.google.com/github/Sobhan-Gh/map-processor/blob/main/Map_to_N_photon_density_matrix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
!pip install qutip
from functools import reduce  # Python 3 forward compatibility
import operator
from qutip import *
from sympy import MatrixSymbol, Matrix


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.7 MB 6.9 MB/s 


In [49]:
#this is needed to access a nested list using a list of indices; e.g. use [1,3,2] to access List[1][3][2]'s element
def access(lst, indexes):
    return reduce(operator.getitem, indexes, lst)

In [50]:
### this is just a variable nested loop function, other functions are derived from this one but are completely different and modified.

def var_for(n,range_list,counter_list):
  #counter list must be set to (n+1)*[0] list before function call. It containt the list of the counters in the reverse order.
  # n is the number of for loops - 1
  # range list is a list of tuples each representing the lower and upper bounds of each of the "n" sums
  # the last member of this list is the range for the first sigma. Below ordering is reveresed to make the input similar to the sum order in sigma notation.
  ordered_range=range_list[::-1]
  if n==0:
    for i in range(ordered_range[n][0],ordered_range[n][1]):
      #change the last counter member in the last loop as the loop spans the range
      counter_list[n]=i
      print(f'the last i equals {i} and (reversed before print) counters equal{counter_list[::-1]}')
    return
  else:
    for i in range(ordered_range[n][0],ordered_range[n][1]):
      counter_list[n]=i
      print(i)
      var_for(n-1,range_list,counter_list)
    return

In [81]:
#Blank density is a zero matrix with enough size for N photons + 1 emitter in pauli basis 

def blank_density(N):
  '''N is the number of photons'''
  a=[]
  if N==0:
    return [0,0,0,0]
  else:
    for i in range(4):
      a.append(blank_density(N-1))
    return a


In [95]:
def element_calculator(n,range_list,counter_list,element_indices):
  #counter list must be set to (n+1)*[0] list before function call. It containt the list of the counters in the reverse order.
  # n is the number of for loops - 1
  # range list is a list of tuples each representing the lower and upper bounds of each of the "n" sums
  # the last member of this list is the range for the first sigma. Below ordering is reveresed to make the input similar to the sum order in sigma notation.
  ordered_range=range_list[::-1]
  if n==0:
    sum=0
    for i in range(ordered_range[n][0],ordered_range[n][1]):
      #change the last counter member in the last loop as the loop spans the range
      counter_list[n]=i
      #print(f'the last i equals {i} and (reversed before print) counters equal{counter_list[::-1]}, element{element_indices}')
      m=1
      c=0
      for a in counter_list:
        if c==0 & len(counter_list)>1:
          m=m*phi[element_indices[0]][element_indices[1]][counter_list[c]]
          c=c+1
        elif c==0 and len(counter_list)==1:
          m=m*phi[element_indices[0]][element_indices[1]][counter_list[c]]*rho0[counter_list[c]]
          c=c+1
        elif c==(len(counter_list)-1):
          m=m*phi[element_indices[c+1]][counter_list[c-1]][counter_list[c]]*rho0[counter_list[c]]
          c=c+1
        else:
          m=m*phi[element_indices[c+1]][counter_list[c-1]][counter_list[c]]
          c=c+1
      sum=sum+m
      #print(f'm={m},sum={sum}')
    return sum
  else:
    s=0
    for i in range(ordered_range[n][0],ordered_range[n][1]):
      counter_list[n]=i
      #print(f's={s}')
      s=s+element_calculator(n-1,range_list,counter_list,element_indices)
    return s

In [89]:
def phi_filler(n,range_list,counter_list):
  #counter list must be set to (n+1)*[0] list before function call. It containt the list of the counters in the reverse order.
  # n is the number of for loops - 1 = here n should be same as N = number of photons
  # range list is a list of tuples each representing the lower and upper bounds of each of the "n" sums
  # the last member of this list is the range for the first sigma. Below ordering is reveresed to make the input similar to the sum order in sigma notation.
  ordered_range=range_list[::-1]
  #to be used as the element calulator function inputs:
  n1=N-1
  range_list1=range_list[:-1]
  counter_list1=counter_list[:-1]
  #
  if n==0:
    for i in range(ordered_range[n][0],ordered_range[n][1]):
      #change the last counter member in the last loop as the loop spans the range
      counter_list[n]=i
      #print(f'the last i equals {i} and (reversed before print) counters equal{counter_list[::-1]}')
      ordered_counter=counter_list[::-1]
      element_indices1=ordered_counter
      the_element=0
      the_element=element_calculator(n1,range_list1,counter_list1,element_indices1)
      #print(f'the element{the_element}')
      #access and assign the nested list's desired element using a list of indices 
      #print(f'element={the_element}')
      density_element_set('density_matrix_decom',element_indices1,the_element)
      #print(f'density-decom{density_matrix_decom}')
    return
  else:
    for i in range(ordered_range[n][0],ordered_range[n][1]):
      counter_list[n]=i
      #print(i)
      phi_filler(n-1,range_list,counter_list)
    return

In [54]:
def state_maker(N,range_list,counter_list):
  index_list=[]
  if N==0:
    for i in range(range_list[N][0],range_list[N][1]):
      counter_list[N]=i
      #print(f'counter {counter_list[::-1]}')
      index_list.append(counter_list[::-1])
    return index_list
  else:
    for i in range(range_list[N][0],range_list[N][1]):
      counter_list[N]=i
      #print(f' {i}')
      index_list.extend(state_maker(N-1,range_list,counter_list))
    return index_list

In [57]:
# a function to access the nested list "density_matrix_decom" (here it is a global list) using the list of index "b=[b0,b1,...]" \ 
# instead of writing density_matrix_decom[b0][b1]...  
def density_element_set(a,b,new_value):
  '''gets the name of a list "a" in string format, and the list of indices b=[b0,b1,...] and sets a[b0][b1][...] = new_value'''
  #a='density_matrix_decom'
  str_list=[f'{a}']
  for x in b:
    str_list.extend([xx for xx in list(f'[{x}]')])
  str_list.extend(['=',f'{new_value}'])
  cc=''.join(str_list)
  exec(cc)
  return

In [ ]:
#Solver for N photons! be wary of gobal variables, they need to be re-assigned before each use

N=5 # set  number of photons

#blank density matrix - it is not a density matrix in fact, but its decomposition in pauli basis- the ordering of DE and photons are as in Gershoni paper: DE-N-(N-1)- ... -2-1
density_matrix_decom=blank_density(N)

#input the process map
#phi = 4*[4*[4*[0.25]]]
phi=[[[0.5,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0.5]],[[0,0,0,0],[0,0.5,0,0],[0,0,0.5,0],[0,0,0,0]],\
     [[0,0,0,-0.5],[0,0,0,0],[0,0,0,0],[-0.5,0,0,0]],[[0,0,0,0],[0,0,0.5,0],[0,-0.5,0,0],[0,0,0,0]]]

#input initial state of DE or emitter
rho0=[0.5,0.5,0,0]

counter_list=(N+1)*[0] # plus one to include the emitter

range_list=[]
for i in range(N+1):
  range_list.append((0,4))

#this function fills out density_matrix_decom
phi_filler(N,range_list,counter_list)

density_matrix_decom

In [ ]:
counter_list=(N+1)*[0]

#a list of indices to access the density matrix decomposition elements to multiple them with corresponding tensor of paulies.
index_list=state_maker(N,range_list,counter_list)
#a single vector of pauli 
s=[qeye(2),sigmax(),sigmay(),sigmaz()]

density_matrix=0
for x in index_list:
  Pauli=0.5*qeye(2)
  for xx in x:
    Pauli=tensor(Pauli,s[xx])
  #remove the first initial identity matrix from the tensor product, to keep is a list of subspaces from 1 to N+1 to keep all photons and emitter
  to_keep=[a for a in range(1,len(x)+1)]
  Pauli=Pauli.ptrace(to_keep)
  density_matrix=density_matrix+access(density_matrix_decom, x)*Pauli
Matrix(density_matrix.ptrace([1,2,3]))

In [ ]:
Matrix(density_matrix)